__собираем данные__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/flat/avito' 
cian_raw_data_path = 'data/raw/flat/cian'

result_file_path = 'data/data_flat.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/flat/avito/avito_2022-08-26_17-29_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-29_13-16_raw.xlsx',
 'data/raw/flat/avito/avito_2022-08-31_17-42_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-01_14-07_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-02_13-04_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-05_14-02_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-06_13-09_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-07_12-39_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-08_13-36_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-12_13-44_raw.xlsx',
 'data/raw/flat/avito/avito_2022-09-13_13-55_raw.xlsx']

In [7]:
from lib.avito import AvitoDataCleanerRealtyFlat

data_avito = pd.concat([
        pd
        .read_excel(f)
        .dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

data_avito = AvitoDataCleanerRealtyFlat().transform( data_avito )

print(len(data_avito))

17951


In [8]:
assert len(data_avito)>0

In [9]:
data_avito.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17951 entries, 0 to 17950
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   avito_id       17951 non-null  int64         
 1   title          17951 non-null  object        
 2   price          17951 non-null  int64         
 3   adr            17951 non-null  object        
 4   description    17951 non-null  object        
 5   obj_name       17951 non-null  object        
 6   ts             17951 non-null  datetime64[ns]
 7   avito_page     17951 non-null  int64         
 8   nrooms         17951 non-null  int64         
 9   floor          17951 non-null  int64         
 10  nfloors        17951 non-null  int64         
 11  area           17951 non-null  float64       
 12  is_studio      17951 non-null  bool          
 13  is_apartment   17951 non-null  bool          
 14  is_part        17951 non-null  bool          
 15  is_auction     1795

In [10]:
# data_avito.query('nrooms==0')
# data_avito[ data_avito['title'].str.len()<1]

## собираем данные циан.ру

In [11]:
import re
from os import listdir

raw_data_files = sorted([
        cian_raw_data_path+'/'+f 
        for f in listdir(cian_raw_data_path) 
        if re.match(r'.+\.pkl$',f)
        #if re.match(r'.+\.xlsx$',f)
    
    ])
raw_data_files

['data/raw/flat/cian/cian_2022-09-08_14-25_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-12_13-59_raw.pkl',
 'data/raw/flat/cian/cian_2022-09-13_13-59_raw.pkl']

In [12]:
data_cian = pd.concat([
        # pd.read_excel(f)
        pd.read_pickle(f)
         .dropna()
         .drop_duplicates(['LinkArea','Description',])
         .rename(columns={'page':'cian_page'})
         .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

In [13]:
data_cian.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4515 entries, 0 to 4514
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   OfferTitle     4515 non-null   object        
 1   OfferSubtitle  4515 non-null   object        
 2   Deadline       4515 non-null   object        
 3   MainPrice      4515 non-null   object        
 4   PriceInfo      4515 non-null   object        
 5   GeoLabel       4515 non-null   object        
 6   TimeLabel      4515 non-null   object        
 7   LinkArea       4515 non-null   object        
 8   Description    4515 non-null   object        
 9   cian_page      4515 non-null   int64         
 10  ts             4515 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 388.1+ KB


In [14]:
swap_title = data_cian['OfferSubtitle'].str.match(r'.*\d+ м², \d+/\d+ эт.*') 

data_cian = pd.concat([
        data_cian[ ~swap_title ],
        data_cian[ swap_title ]
            .rename(columns={'OfferTitle':'OfferSubtitle','OfferSubtitle':'OfferTitle'})
    ]).reset_index(drop=True)


In [15]:
# data_cian

In [16]:
from lib.cian import CianDataCleaner

data_cian = CianDataCleaner().transform( data_cian )

print(len(data_cian))

4515


In [17]:
assert len(data_cian)>0

In [18]:
data_cian.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4515 entries, 0 to 4514
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          4515 non-null   object        
 1   obj_name       4515 non-null   object        
 2   adr            4515 non-null   object        
 3   nrooms         4515 non-null   int64         
 4   floor          4515 non-null   int64         
 5   nfloors        4515 non-null   int64         
 6   area           4515 non-null   float64       
 7   is_studio      4515 non-null   bool          
 8   is_apartment   4515 non-null   bool          
 9   is_part        4515 non-null   bool          
 10  is_auction     4515 non-null   bool          
 11  is_openspace   4515 non-null   bool          
 12  is_roof        4515 non-null   bool          
 13  is_SNT         4515 non-null   bool          
 14  price          4515 non-null   int64         
 15  priceM         4515 n

In [19]:
# data_cian
# data_cian.query('nrooms==0')
# data_cian[ data_cian['title'].str.len()<1 ]
# data_cian[ data_cian['obj_name'].str.len()>0 ][['title','obj_name',]] # .to_csv('tmp/cian_title.csv',sep='\t')|

----

In [20]:
data = pd.concat([data_avito,data_cian]).reset_index(drop=True) 
print(len(data))
assert len(data)>0

22466


### обновляем таблицу адресов

In [ ]:
from lib.locator import LocationUpdater
from lib.locator import AddressTransformerSev
from lib.locator import GeocoderYandex
from yandex_keys import keys

loc_file_path = 'data/location_yandex.pkl'# таблица адресов

loc = LocationUpdater(
        locator=GeocoderYandex(keys[0]),
        address_transformer=AddressTransformerSev(),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

[INFO    ] 2022-09-13 15:36:31 | LocationUpdater: 875 addresses in location table
[INFO    ] 2022-09-13 15:36:31 | LocationUpdater: 3639 addresses total
[INFO    ] 2022-09-13 15:36:31 | LocationUpdater: 875 addresses defined
[INFO    ] 2022-09-13 15:36:31 | LocationUpdater: 2764 addresses undefined


  0%|          | 0/2764 [00:00<?, ?it/s]

In [ ]:
# from lib.locator import LocationUpdater
# from lib.locator import AddressTransformerSev

# loc_file_path = 'data/location_osm.pkl'# таблица адресов

# loc = LocationUpdater(
#         address_transformer=AddressTransformerSev(),
#     ).transform(
#         adr=data['adr'],
#         loc=pd.read_pickle(loc_file_path+'/location_osm.pkl'),
#         show_pbar=True,
#     )

In [ ]:
loc.sample(3)

In [ ]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

### дополняем данные геометкой

In [ ]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

In [ ]:
assert len(data)>0
data.to_pickle(result_file_path)

In [ ]:
data.info()

In [ ]:
# for f in raw_data_files:
#     ts = dtm.strptime( re.sub(r'.*/avito_','',f), '%Y-%m-%d_%H-%M_raw.xlsx')
#     df = pd.read_excel(f)
#     df['ts'] = ts
#     df.to_excel(f,index=False)
#     print(ts)